In [ ]:
# coding:utf-8

# setup: websocket open:8888
#sudo pip install findspark
#sudo pip install ipython==5.0
#sudo pip install jupyter
#sudo pip install matplotlib
#sudo pip install scipy
#history|grep "pip install"

# set encoding, set print stdinout
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

# spark context
from pyspark import SparkContext
from pyspark import SparkConf
conf = SparkConf()
sc = SparkContext.getOrCreate(conf)

# parse json
import json

# use matplotlib
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
%matplotlib inline

plt.figure(figsize=(10,10))

#plot scatter, hist, plot
#plt.scatter(displayBar, pctrBar)
#plt.plot(pctrTime, pctrBar)
#plt.hist(X, 10)
#plt.plot(p2, "r")

def calcCalib(data5):
    data6 = data5.sortBy(lambda x: x[0])
    bid = data6.map(lambda x:x[0]).collect()
    p = data6.map(lambda x:x[1]).collect()
    l = data6.map(lambda x:x[2]).collect()
    temp = 0
    temp2 = 0
    p2 = []
    p3 = []
    p4 = []
    for i,pp in enumerate(zip(p,bid)):
        temp += pp[0]
        p2.append(temp*pp[1] / (i+1))
        p3.append(temp / (i+1))
        p4.append(pp[1] * 0.02)

    temp = 0
    l2 = []
    for i,ll in enumerate(l):
        temp += ll
        l2.append(temp / (i+1))

    calibratio = 1.0
    if p3[-1] < l2[-1]:
        calibratio = 1.01
    elif p3[-1] > l2[-1]:
        for i in xrange(0, len(p2)):
            if (p2[-i-1] <= l2[-i-1]):
                calibratio = bid[-i-1]
                break
    
    if calibratio < 0.3:
        calibratio = 0.3
    
    #plt.plot(bid)
    fig = plt.figure(figsize=(10,10)) 
    plt.ylim([0,0.02])
    plt.plot(p2, "r")
    plt.plot(l2, "g")
    plt.plot(p3, "b")
    plt.plot(p4, "y")

    fig.show()
    print target, "oe:", p3[-1]/l2[-1], "calibratio:", calibratio